# Сверточные нейронные сети (CNN)

**Обозначения**:
- Значение в квадратных скобках $[l]$ обозначает, что объект принадлежит слою l. 
    - Например, $a^{[4]}$ - функция активации четвертого слоя. $W^{[5]}$ и $b^{[5]}$ - это параметры 5-го слоя.


- Обозначение $(i)$ говорит о принадлежности объекта к $i-тому$ входному тензору. 
    - Например, $x^{(i)}$ означает $i-тый$ входной тензор из датасета (например, i-я картинка).
    
    
- Индекс $i$ обозначает $i-тый$ элемент вектора.
    - Например, $a^{[l]}_i$ означает $i-тое$ значение функции активации $l-того$ слоя.
    
    
- $n_H$, $n_W$ и $n_C$ обозначают высоту, ширину и количество каналов данного слоя. Говоря о конкретном слое $l$, можно написать $n_H^{[l]}$, $n_W^{[l]}$, $n_C^{[l]}$. 
- $n_{H_{prev}}$, $n_{W_{prev}}$ и $n_{C_{prev}}$ обозначают высоту, ширину и количество каналов предыдущего слоя. Говоря о конкретном слое $l-1$, можно написать $n_H^{[l-1]}$, $n_W^{[l-1]}$, $n_C^{[l-1]}$. 



<a name='1'></a>
## 1 - Импорт библиотек

- [numpy](www.numpy.org) базовая библиотека для работы с массивами.
- [matplotlib](http://matplotlib.org) используется для визуализации графиков.
- np.random.seed(1) устанавливает начальное значение для псевдослучайного генератора. Позволяет добиться повторяемости выполнения кода.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # устанавливаем размер графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

<a name='2'></a>
## 2 - План

Нам необходимо реализовать базовые блоки, из которых затем собирается сверточная нейронная сеть. Эти упражнения помогут нам наиболее глубоко понять механику работы (математику) слоев.

- Что необходимо реализовать:
    - Zero Padding
    - Convolve window 
    - Convolution forward

Реализовывать эти функции в основном будем при помощи библиотеки numpy.

<a name='3'></a>
## 3 - Приступаем к реализации

<a name='3-1'></a>
### 3.1 - Zero-Padding

Zero-padding добавляет нули по краям изображения:

<caption><center> <img src="images/PAD.png" style="width:600px;height:400px;"> </center></caption>
<caption><center> <u> <b>Рисунок 1</b>  : <b>Zero-Padding</b> <br> Изображение (3 канала, RGB) с дополнением 2 нулями.</br> </center></caption>


Что нам дает padding:

- Позволяет избежать снижения значений размерностей из-за операции свертки. 

- Позволяет ядрам свертки чаще учитывать информацию с краев изображения.

Необходимо реализовать функцию, которая дополняет все тензоры, хранящиеся в массиве X, нулями. [Используйте функцию np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html). 

Примечание: для дополнения массива "a" размера $(5,5,5,5,5)$ с параметрами `pad = 1` для размерности 1, `pad = 3` для размерности 3 и `pad = 0` для остальных размерностей, код будет выглядеть следующим образом:
```python
a = np.pad(a, ((0,0), (1,1), (0,0), (3,3), (0,0)), mode='constant', constant_values = (0,0))
```

In [ ]:

def zero_pad(X, pad):
    """
    Аргументы:
    X -- массив с размерность. (m, n_H, n_W, n_C), содержит m тензоров, представляющих изображения
    pad -- целое число, показывает, сколько нулей должно быть добавлено по краям изображения
    
    Возвращает:
    X_pad -- изображение с размерностю (m, n_H + 2 * pad, n_W + 2 * pad, n_C)
    """
    
    # Дополните код
    X_pad = ...
    # -------------
    
    return X_pad

In [ ]:
np.random.seed(1)
x = np.random.randn(4, 3, 3, 2)
x_pad = zero_pad(x, 3)
print ("x.shape =\n", x.shape)
print ("x_pad.shape =\n", x_pad.shape)
print ("x[1,1] =\n", x[1, 1])
print ("x_pad[1,1] =\n", x_pad[1, 1])

fig, axarr = plt.subplots(1, 2)
axarr[0].set_title('x')
axarr[0].imshow(x[0, :, :, 0])
axarr[1].set_title('x_pad')
axarr[1].imshow(x_pad[0, :, :, 0])

<a name='3-2'></a>
### 3.2 - Шаг свертки

В данном задании необходимо имплементировать один шаг операции свертки, в котором фильтр накладывается на одну позицию входного тензора. Далее будет реализована полноценная операция свертки, в которую входит сдвиг фильтра, выполнение шага свертки, затем снова сдвиг фильтра и так далее.

<caption><center><img src="images/Convolution_schematic.gif" style="width:500px;height:300px;"></caption></center>
<caption><center> <u> <b>Рисунок 2</b> </u> : <b>Операция свертки</b><br> с ядром размера 3x3 и сдвигом на одну позицию </center></caption>

**Примечание**: Переменная b является массивом numpy. Если прибавить к массиву скаляр, в результате этот скаляр прибавится ко всем элементам массива.

In [ ]:
def conv_single_step(a_slice_prev, W, b):
    """
    Аргументы:
    a_slice_prev -- входной тензор размера (f, f, n_C_prev)
    W -- Веса фильтра -матрица размера (f, f, n_C_prev)
    b -- Значение сдвига (bias) - матрица размера (1, 1, 1)
    
    Возвращает:
    Z -- скаляр, результат свертки (W, b) и входных данных
    """

    # Дополните код

    # Поэлементное произведение a_slice_prev и весов
    s = ...
    # Сумма элементов произведения
    Z = ...
    # Добавьте значение сдвига b к сумме Z. Можно преобразовать b к виду float().
    Z = ...
    # -----------------

    return Z

In [ ]:
np.random.seed(1)
a_slice_prev = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 1)

Z = conv_single_step(a_slice_prev, W, b)
print("Z =", Z)

assert (type(Z) == np.float64), "Выходное значение должно иметь тип данных float 64"
assert np.isclose(Z, -6.999089450680221), "Неверный результат!"

<a name='3-3'></a>
### 3.3 - Сверточный слой нейронной сети

В этом задании необходимо выполнить полноценную свертку нескольких фильтров с входным тензором. Каждая 'свертка' даст двумерную матрицу на выходе. Эти матрицы далее необходимо объединить в трехмерный массив. 

<center>
<video width="620" height="440" src="images/conv_kiank.mp4" type="video/mp4" controls>
</video>
</center>


Функция выполняет свертку весов `W` с выходом функции активации предыдущего слоя `A_prev`.  
Входные значения функции:
* `A_prev`, выходы функции активации предыдущего слоя (батч имеет размер m); 
* Матрицы весов фильтров `W`.  Размер окна фильтра (ядра свертки) `f` на `f`.
* Вектор значений сдвига `b`, где для каждого фильтра используется свое значение сдвига. 

Также в функцию подается словарь с гиперпараметрами, которые содержат значения stride и padding. 

**Примечания**: 
1. Для того, чтобы выбрать часть массива (левый верхний угол) размера 2x2 из матрицы "a_prev" (размера (5,5,3)), можно выполнить команду:
```python
a_slice_prev = a_prev[0:2,0:2,:]
```
Здесь третья размерность - размерность каналов, не указывая значения диапазона, а оставляя двоеточие, мы можем выбрать весь диапазон.  
Это пригодится при формировании `a_slice_prev` совместно с использованием индексов `start/end`.

2. Выбор части массива задается через координаты его углов `vert_start`, `vert_end`, `horiz_start` и `horiz_end`. Картинка может помочь понять, как задаются значения h, w, f и s в коде далее.

<caption><center> <img src="images/vert_horiz_kiank.png" style="width:400px;height:300px;"></center></caption>
<caption><center>  <b>Рисунок 3</b> : <b>Задание координат для получения доступа к части массива (для одного канала) </center></caption>


**Напоминание**:
    
Выходные размерности свертки задаются следующими формулами:
    
$$n_H = \Bigl\lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_W = \Bigl\lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_C = \text{количество фильтров в свертке}$$
    

In [ ]:

def conv_forward(A_prev, W, b, hparameters):
    """
    Arguments:
    A_prev -- выход функции активации предыдущего слоя, 
        массив Numpy  с размерностями (m, n_H_prev, n_W_prev, n_C_prev)
    W -- веса фильтров с размерностями (f, f, n_C_prev, n_C)
    b -- вектор значений сдвигов с размерностями (1, 1, 1, n_C)
    hparameters -- словарь со значениями "stride" и "pad"
        
    Выход функции:
    Z -- результат свертки размерности (m, n_H, n_W, n_C)
    """
    
    # Получаем размерности из массива A_prev 
    (m, n_H_prev, n_W_prev, n_C_prev) = ...
    
    # Получаем размерности из массива W
    (f, f, n_C_prev, n_C) = ...
    
    # Получаем значения из словаря гиперпараметров
    stride = ...
    pad = ...

    
    # Вычисляем выходные размерности операции свертки (формулы даны выше). 
    # Подсказка: используйте функцию int() для округления вниз 
    n_H = ...
    n_W = ...
    
    # Инициализация выходного массива Z нулями
    Z = ...
    
    # Создайте массив A_prev_pad при помощи дополнения нулями массива A_prev
    A_prev_pad = ...
    
    for i in range(...):               # цикл по батчам (за раз в функцию подается m тензоров)
        a_prev_pad = ...               # Выбор i-того тензора, являющегося выходом предыдущий функции активации (после дополнения нулями)
        for h in range(...):           # цикл по столбцам тензора
            #Вычисление начала и конца (по вертикали) для необходимой части тензора
            vert_start = ...
            vert_end = ...
            
            for w in range(...):       #цикл по строкам тензора
                #Вычисление начала и конца (по горизонтали) для необходимой части тензора
                horiz_start = ...
                horiz_end = ...
                
                for c in range(...):   # цикл по каналам выходного тензора (количество каналов = количеству фильтров)
                                        
                    #Формирование трехмерной подматрицы матрицы a_prev_pad при помощи найденных углов
                    a_slice_prev = ...
                    
                    #Свертка трехмерной подматрицы a_slice_prev с нужным фильтром W и значением сдвига.
                    weights = ...
                    biases = ...
                    Z[i, h, w, c] = ...
    
    return Z

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(2, 5, 7, 4)
W = np.random.randn(3, 3, 4, 8)
b = np.random.randn(1, 1, 1, 8)
hparameters = {"pad" : 1,
               "stride": 2}

Z = conv_forward(A_prev, W, b, hparameters)
z_mean = np.mean(Z)
z_0_2_1 = Z[0, 2, 1]
print("Z's mean =\n", z_mean)
print("Z[0,2,1] =\n", z_0_2_1)

# Z's mean =
#  0.5511276474566768
# Z[0,2,1] =
#  [-2.17796037  8.07171329 -0.5772704   3.36286738  4.48113645 -2.89198428
#  10.99288867  3.03171932]


Наконец, к выходу операции свертки добавляется выполнение функции активации. Для этого необходимо внести небольшое изменение:

```python
# Операция свертки, реализованная ранее
Z[i, h, w, c] = ...
# Применение функции активации
A[i, h, w, c] = activation(Z[i, h, w, c])
```


<a name='3.4'></a>
## 3.4 - Pooling Layer 

Слой подвыборки (pooling layer) уменьшает размерности входного тензора (длину/ширину/глубину). Это помогает снизить количество вычислений а также сделать нейронную сеть более устойчивой к изменения положения объекта на входном изображении. Существует два основных вида слоя подвыборки:

- Max-pooling layer: накладывает окно размером ($f, f$) на входной тензор и передает на выход максимальное значение, попавшее в окно.

- Average-pooling layer: накладывает окно размером ($f, f$) на входной тензор и передает на выход среднее по всем значениям, попавшим в окно.

<table>
<td>
<img src="images/max_pool1.png" style="width:500px;height:300px;">
<td>

<td>
<img src="images/a_pool.png" style="width:500px;height:300px;">
<td>
</table>

У данного слоя нет праветров, изменяющихся при обучении. Однако для него устанавливается гиперпраметр - размер окна  $f$. Он определяет высоту и ширину $f \times f$ окна, внутри которого будет вычисляться *максимальное* или *среднее* значение. Также устанавливается размер шага наложения фильтра *stride*.


Напишите слой подвыборки. Выходные размерности слоя определяются следующим образом:

$$n_H = \Bigl\lfloor \frac{n_{H_{prev}} - f}{stride} \Bigr\rfloor +1$$

$$n_W = \Bigl\lfloor \frac{n_{W_{prev}} - f}{stride} \Bigr\rfloor +1$$

$$n_C = n_{C_{prev}}$$

In [ ]:
def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Аргументы:
    A_prev -- Входной тензор, массив numpy размером (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- словарь, содержит параметры "f" и "stride"
    mode -- вид подвыборки, строка, принимает значения ("max" или "average")
    
    Возвращает:
    A -- выходной тензор, массив numpy размером (m, n_H, n_W, n_C)
    """
    
    # Получение размерностей входного тензора
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Получение гиперпараметров из словаря "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Вычисление выходных размерностей
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    # Инициализация матрицы A
    A = np.zeros((m, n_H, n_W, n_C))              
    
    for i in range(None):                         # цикл по батчам (за раз в функцию подается m тензоров)
        for h in range(None):                     # цикл по столбцам матрицы
            #Поиск начала и конца текущей подматрицы по вертикали
            vert_start = None
            vert_end = None
            
            for w in range(None):                 # цикл по строкам матрицы
                #Поиск начала и конца текущей подматрицы по горизонтали
                horiz_start = None
                horiz_end = None
                
                for c in range (None):            # цикл по каналам выходной матрицы
                    
                    #Выбор i-го батча из A_prev, затем выбор подматрицы с углами из vert_start, vert_end, horiz_start, horiz_end
                    a_prev_slice = None
                    
                    #Выполнение операции подвыборки (pooling)  
                    #Использвуйте np.max and np.mean.
                    if mode == "max":
                        A[i, h, w, c] = None
                    elif mode == "average":
                        A[i, h, w, c] = None
                        
    # Проверка, что выходные размерности корректны
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A

In [ ]:
# Проверка 1: stride = 1
np.random.seed(1)
A_prev = np.random.randn(2, 5, 5, 3)
hparameters = {"stride" : 1, "f": 3}

A = pool_forward(A_prev, hparameters, mode = "max")
print("mode = max")
print("A.shape = " + str(A.shape))
print("A[1, 1] =\n", A[1, 1])
A = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A.shape = " + str(A.shape))
print("A[1, 1] =\n", A[1, 1])

**Ожидаемый результат**

```
mode = max
A.shape = (2, 3, 3, 3)
A[1, 1] =
 [[1.96710175 0.84616065 1.27375593]
 [1.96710175 0.84616065 1.23616403]
 [1.62765075 1.12141771 1.2245077 ]]

mode = average
A.shape = (2, 3, 3, 3)
A[1, 1] =
 [[ 0.44497696 -0.00261695 -0.31040307]
 [ 0.50811474 -0.23493734 -0.23961183]
 [ 0.11872677  0.17255229 -0.22112197]]
```

In [ ]:
# Проверка 2: stride = 2
np.random.seed(1)
A_prev = np.random.randn(2, 5, 5, 3)
hparameters = {"stride" : 2, "f": 3}

A = pool_forward(A_prev, hparameters)
print("mode = max")
print("A.shape = " + str(A.shape))
print("A[0] =\n", A[0])
print()

A = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A.shape = " + str(A.shape))
print("A[1] =\n", A[1])

**Ожидаемый результат:**
    
```
mode = max
A.shape = (2, 2, 2, 3)
A[0] =
 [[[1.74481176 0.90159072 1.65980218]
  [1.74481176 1.6924546  1.65980218]]

 [[1.13162939 1.51981682 2.18557541]
  [1.13162939 1.6924546  2.18557541]]]

mode = average
A.shape = (2, 2, 2, 3)
A[1] =
 [[[-0.17313416  0.32377198 -0.34317572]
  [ 0.02030094  0.14141479 -0.01231585]]

 [[ 0.42944926  0.08446996 -0.27290905]
  [ 0.15077452  0.28911175  0.00123239]]]
```